# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


# Importing Libraries

In [2]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [3]:
# Use this cell for any import statements you add



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [4]:
artists = {'taylor':"https://www.azlyrics.com/t/taylorswift.html",
           'ariana': "https://www.azlyrics.com/a/arianagrande.html"}
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: Yes, data scraping is allowed on the site. The 'Allow: /' rule apllies to all parts of the site that are not /lyricsdb/ and /song/, indiciating web scraping is technically allowed.


In [5]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    soup = BeautifulSoup(r.text, 'html.parser')

    div = soup.find('div', {'class': 'col-xs-12 col-lg-8 text-center'})

    if div:
        # All lyrics links have a specific structure like '/lyrics/artist/songtitle.html'
        for link in div.find_all('a', href=True):
            # Check if the link is a lyrics page (usually contains '/lyrics/')
            if "/lyrics/" in link['href']:
                # Complete the relative URL if necessary and store in lyrics_pages
                lyrics_pages[artist].append("https://www.azlyrics.com" + link['href'])

# Print or inspect the lyrics pages
for artist, links in lyrics_pages.items():
    print(f"{artist}: {len(links)} lyrics links found")
    for link in links:
        print(link)

taylor: 468 lyrics links found
https://www.azlyrics.com/lyrics/taylorswift/samegirl.html
https://www.azlyrics.com/lyrics/taylorswift/babyblue.html
https://www.azlyrics.com/lyrics/taylorswift/neverfade.html
https://www.azlyrics.com/lyrics/taylorswift/fire.html
https://www.azlyrics.com/lyrics/taylorswift/maryjo.html
https://www.azlyrics.com/lyrics/taylorswift/myturntobeme.html
https://www.azlyrics.com/lyrics/taylorswift/kidinthecrowd.html
https://www.azlyrics.com/lyrics/taylorswift/iusedtofly.html
https://www.azlyrics.com/lyrics/taylorswift/luckyyou.html
https://www.azlyrics.com/lyrics/taylorswift/pointofview.html
https://www.azlyrics.com/lyrics/taylorswift/whydoyoutellme.html
https://www.azlyrics.com/lyrics/taylorswift/houstonrodeo.html
https://www.azlyrics.com/lyrics/taylorswift/alittlemorelikeyou.html
https://www.azlyrics.com/lyrics/taylorswift/onesidedgoodbye.html
https://www.azlyrics.com/lyrics/taylorswift/rideon.html
https://www.azlyrics.com/lyrics/taylorswift/smokeyblacknights.htm

Let's make sure we have enough lyrics pages to scrape.

In [6]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20)

In [7]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, links in lyrics_pages.items() :
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For taylor we have 468.
The full pull will take for this artist will take 1.3 hours.
For ariana we have 191.
The full pull will take for this artist will take 0.53 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [8]:
def generate_filename_from_link(link) :

    if not link :
        return None

    # drop the http or https and the html
    name = link.replace("https://", "").replace("http://", "").replace(".html", "")

    # Replace '/lyrics/' with empty string
    name = name.replace("/lyrics/", "")

    # Replace unnecessary characters with an underscore
    name = name.replace(".", "_").replace("/", "_")

    # Append '.txt' to the final name
    name = name + ".txt"

    return name


In [9]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") :
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [12]:
def generate_filename_from_link(url):
    # Basic implementation; adapt as needed
    return url.split('/')[-1].replace('.html', '.txt')

url_stub = "https://www.azlyrics.com"

# Initialize artists with their URLs
artists = {
    'taylor': "https://www.azlyrics.com/t/taylorswift.html",
    'ariana': "https://www.azlyrics.com/a/arianagrande.html"
}

total_pages = 0

for artist, artist_url in artists.items():
    artist_folder = os.path.join("lyrics", artist)
    if not os.path.exists(artist_folder):
        os.makedirs(artist_folder)

    # Fetch the artist's main page
    try:
        r = requests.get(artist_url)
        r.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching {artist_url}: {e}")
        continue

    time.sleep(5 + 10 * random.random())

    # Parse the artist's page to find song links
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find all song links on the page
    song_links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and href.startswith('/lyrics/'):
            song_links.append(href)

    # Iterate over the first 20 lyrics pages
    for i, lyrics_link in enumerate(song_links):
        if i >= 20:  # limit to 20 songs
            break

        # Clean up lyrics_link to avoid issues
        lyrics_link = lyrics_link.lstrip('/')  # Remove any leading slashes
        lyrics_page_url = f"{url_stub}/{lyrics_link}"  # Ensure correct URL formation

        # Request the lyrics page and sleep
        try:
            r = requests.get(lyrics_page_url)
            r.raise_for_status()
        except requests.RequestException as e:
            print(f"Error fetching {lyrics_page_url}: {e}")
            continue

        time.sleep(5 + 10 * random.random())

        # Parse the lyrics page
        soup = BeautifulSoup(r.text, 'html.parser')

        # Extract song title (in <b> tags)
        title = soup.find('b').text.strip() if soup.find('b') else "Unknown Title"

        # Extract lyrics (in <div> without class or id)
        lyrics_div = soup.find('div', class_=None, id=None)
        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
        else:
            lyrics = "Lyrics not found"

        # Generate filename and write to text file
        filename = generate_filename_from_link(lyrics_page_url)
        file_path = os.path.join(artist_folder, filename)

        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(title + "\n\n" + lyrics)
        except IOError as e:
            print(f"Error writing to {file_path}: {e}")
            continue

        total_pages += 1

print(f"Total pages processed: {total_pages}")

Total pages processed: 40


In [13]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.15 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [14]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text):
    return re.findall(r'\w+', text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.

In [18]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir(os.path.join("lyrics", f))]


for artist in artist_folders:
    artist_path = os.path.join("lyrics", artist)
    artist_files = os.listdir(artist_path)
    artist_files = [f for f in artist_files if f.endswith(('txt', 'csv', 'tsv'))]

    
    if artist_files:
        print(f"For {artist} we have {len(artist_files)} files.")


For taylor we have 20 files.
For ariana we have 20 files.
